In [1]:
data_dir = './data/mlboot_dataset/'
results_dir = './results/'
import pandas as pd
train = pd.read_table(data_dir + 'mlboot_data.tsv', header=None, names=['uid','cat','f1','f2','f3','days'], usecols=['uid','cat','days'])

KeyboardInterrupt: 

In [ ]:
def num_unique_keys(s):
    d = eval(s)
    return len(d.keys())
def sum_values(s):
    d = eval(s)
    return sum(d.values())
def max_value(s):
    d = eval(s)
    if len(d.keys()) > 0:
        return max(d.values())
    return 0

for f in ['f1','f2','f3']:
    train['num_keys_'+f] = train[f].apply(num_unique_keys)
    train['sum_values_'+f] = train[f].apply(sum_values)

for cc in range(6):
    qq = train.loc[train.cat == cc,['uid','cat']].groupby('uid').agg('count').reset_index()
    qq.columns = ['uid','num_times_cat_eq_'+str(cc)]
    train = train.merge(qq, on='uid', how='left')

q = train[['uid','cat']].groupby('uid').agg('count').reset_index()
q.columns = ['uid','records']
train = train.merge(q, on='uid', how='left')

qq = train[['uid','days']].groupby('uid').agg('max').reset_index()
qq.columns = ['uid','max_days']
train = train.merge(qq, on='uid', how='left')

qq = train[['uid','days']].groupby('uid').agg('min').reset_index()
qq.columns = ['uid','min_days']
train = train.merge(qq, on='uid', how='left')

for fun in ['std','max','mean']:
    for f in ['f1','f2','f3']:
        for c in ['sum_values_','num_keys_']:
            col = c+f
            qq = train[['uid',col]].groupby('uid').agg(fun).reset_index()
            qq.columns = ['uid',col+'_'+fun]
            train = train.merge(qq, on='uid', how='left')

df = train[[u'uid', u'cat', u'num_keys_f1',
       u'sum_values_f1', u'num_keys_f2', u'sum_values_f2', u'num_keys_f3',
       u'sum_values_f3', u'num_times_cat_eq_0', u'num_times_cat_eq_1',
       u'num_times_cat_eq_2', u'num_times_cat_eq_3', u'num_times_cat_eq_4',
       u'num_times_cat_eq_5', u'records', u'max_days', u'min_days',
       u'sum_values_f1_std', u'num_keys_f1_std', u'sum_values_f2_std',
       u'num_keys_f2_std', u'sum_values_f3_std', u'num_keys_f3_std',
       u'sum_values_f1_max', u'num_keys_f1_max', u'sum_values_f2_max',
       u'num_keys_f2_max', u'sum_values_f3_max', u'num_keys_f3_max',
       u'sum_values_f1_mean', u'num_keys_f1_mean', u'sum_values_f2_mean',
       u'num_keys_f2_mean', u'sum_values_f3_mean', u'num_keys_f3_mean']].copy()
df.drop_duplicates(subset='uid',inplace=True)
df.to_csv(data_dir+'preprocessed.csv',index=False)            

In [3]:
data_dir = './data/mlboot_dataset/'
results_dir = './results/'
import pandas as pd

train = pd.read_table(data_dir + 'mlboot_data.tsv', header=None, names=['uid','cat','f1','f2','f3','days'], usecols=['uid','f1','f2','f3'])
import scipy.sparse as sp
import numpy as np

mat1 = sp.lil_matrix((len(train),2053602), dtype=np.int32)
row = 0
def process_df(df):
    global mat1, row
    d = eval(df)
    for i,q in d.items():
        mat1[row,int(i)] += q
    row += 1
    if row % 1000000 == 0:
        print(row,':',len(train))
    return
train.f1.apply(process_df)
sp.save_npz(data_dir+'mat1.npz', mat1.tocsr())

mat2 = sp.lil_matrix((len(train),2812610), dtype=np.int32)
row = 0
def process_df(df):
    global mat2, row
    d = eval(df)
    for i,q in d.items():
        mat2[row,int(i)] += q
    row += 1
    if row % 1000000 == 0:
        print(row,':',len(train))    
    return
train.f2.apply(process_df)
mat2 = mat2.tocsc()[:, np.where(mat2.getnnz(axis=0) > 0)[0]].tocsr()
sp.save_npz(data_dir+'mat2.npz', mat2.tocsr())


mat3 = sp.lil_matrix((len(train),1057788), dtype=np.int32)
row = 0
def process_df(df):
    global mat3, row
    d = eval(df)
    for i,q in d.items():
        mat3[row,int(i)] += q
    row += 1
    if row % 1000000 == 0:
        print(row)     
    return
train.f3.apply(process_df)
sp.save_npz(data_dir+'mat3.npz', mat3.tocsr())

1000000 : 19528597
2000000 : 19528597
3000000 : 19528597
4000000 : 19528597
5000000 : 19528597
6000000 : 19528597
7000000 : 19528597
8000000 : 19528597
9000000 : 19528597
10000000 : 19528597
11000000 : 19528597
12000000 : 19528597
13000000 : 19528597
14000000 : 19528597
15000000 : 19528597
16000000 : 19528597
17000000 : 19528597
18000000 : 19528597
19000000 : 19528597
1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
11000000
12000000
13000000
14000000
15000000
16000000
17000000
18000000
19000000


In [4]:
import tqdm

lunique = train.uid.unique()
N = len(train)
M = len(lunique)
mask=sp.lil_matrix((M,N), dtype=np.int32)
row = -1
i = 0
last_uid = 'a'
for cuid in tqdm.tqdm(train.uid):
    if last_uid != cuid:
        last_uid = cuid
        row += 1
    mask[row,i] = 1
    i = i + 1
    
mat1 = sp.load_npz(data_dir+'mat1.npz')
mat1 = (mask*mat1).tocsr()
mat1.shape

mat2 = sp.load_npz(data_dir+'mat2.npz')
mat2 = (mask*mat2).tocsr()
mat2.shape

mat3 = sp.load_npz(data_dir+'mat3.npz')
mat3 = (mask*mat3).tocsr()
mat3.shape

sp.save_npz(data_dir+'dmat1.npz', mat1.tocsr())
sp.save_npz(data_dir+'dmat2.npz', mat2.tocsr())
sp.save_npz(data_dir+'dmat3.npz', mat3.tocsr())

sp.save_npz(data_dir+'bmat1.npz', mat1.tocsr().astype(np.bool).astype(np.int8))
sp.save_npz(data_dir+'bmat2.npz', mat2.tocsr().astype(np.bool).astype(np.int8))
sp.save_npz(data_dir+'bmat3.npz', mat3.tocsr().astype(np.bool).astype(np.int8))

100%|██████████| 19528597/19528597 [01:29<00:00, 218181.08it/s]
